In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
csv_path = 'C:\\Users\\SIU856533724\\OneDrive - Southern Illinois University\\Research\\Paper\\Social Networks\\Trend Prediction\\Comment-Section-UI\\UI\\angular-interactive-comments-section\\src\\data\\'

C:\Users\SIU856533724\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
file = os.path.join(csv_path, 'fox-college-students-threatened-punishment.csv')
df = pd.read_csv(file)

In [3]:
df.head(20)

,Unnamed: 0,comment,time,username,id,timestamp,time_diff
0,64,Hate needs to be stopped.,"5 Dec, 2023 04:20 AM",anakalamaui,65,2023-12-05 04:20:00,0 days 00:00:00
1,0,It is so easy to fix this problem. There's a f...,"5 Dec, 2023 05:50 AM",O's,1,2023-12-05 05:50:00,0 days 01:30:00
2,13,"""To learn who rules over you, simply find out ...","5 Dec, 2023 06:02 AM",Enlightened672,14,2023-12-05 06:02:00,0 days 01:42:00
3,52,I support Gaza because God is not about oppres...,"5 Dec, 2023 06:02 AM",ripgop246,53,2023-12-05 06:02:00,0 days 01:42:00
4,51,Should be allowed to say whatever you want You...,"5 Dec, 2023 06:03 AM",stillunrepresented,52,2023-12-05 06:03:00,0 days 01:43:00
5,42,Free speech is only an issue when it involves ...,"5 Dec, 2023 06:12 AM",peace1,43,2023-12-05 06:12:00,0 days 01:52:00
6,40,Anti-American liberals absolutely hate the Con...,"5 Dec, 2023 06:13 AM",Dilligaf-666,41,2023-12-05 06:13:00,0 days 01:53:00
7,69,Euthanize palestine.,"5 Dec, 2023 06:15 AM",XOXIDENOFCHINA,70,2023-12-05 06:15:00,0 days 01:55:00
8,50,This article sure makes me want to spend tax d...,"5 Dec, 2023 06:20 AM",NotMarkOwens,51,2023-12-05 06:20:00,0 days 02:00:00
9,16,And as long as censorial administrators have d...,"5 Dec, 2023 06:31 AM",2cents1968,17,2023-12-05 06:31:00,0 days 02:11:00


In [4]:
list(df.iloc[1][1:])

["It is so easy to fix this problem. There's a fast fix and a slow fix. The fast fix is to eliminate the guaranteed student loan program. This will force higher learning to compete for customers, aka students. Many institutions will not make it. The slow fix is to identify institutions that demonstrate freedom and send your kids there. Again, this will force institutions to compete for customers, but it will take more time to see results. Higher education is a business, not unlike Bud Light. Treat it accordingly if you want to make a point.",
 '5 Dec, 2023 05:50 AM',
 "O's",
 1,
 '2023-12-05 05:50:00',
 '0 days 01:30:00']

In [5]:
def get_so_far_comments(comment_id):
    row_index = df.loc[df['id'] == comment_id].index[0]
    ids = df.iloc[0:row_index+1]['id']
    return list(ids)


In [6]:
get_so_far_comments(1)

[65, 1]

In [7]:
from score_aggregation import get_emotion_trends_polarity

def getData(commentId):
    ids = get_so_far_comments(comment_id=commentId)
    print("Id: ", commentId)
    print("So far Comments: ", ids)
    escores, tscores, polarity = get_emotion_trends_polarity(ids)
    return escores, tscores, polarity

{'positive': ['amusement', 'excitement', 'joy', 'love', 'desire', 'optimism', 'caring', 'pride', 'admiration', 'gratitude', 'relief', 'approval'], 'negative': ['fear', 'nervousness', 'remorse', 'embarrassment', 'disappointment', 'sadness', 'grief', 'disgust', 'anger', 'annoyance', 'disapproval'], 'ambiguous': ['realization', 'surprise', 'curiosity', 'confusion'], 'neutral': ['neutral']}
dict_keys(['positive', 'negative', 'ambiguous', 'neutral'])


In [8]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import json

app = Flask(__name__)
cors = CORS(app, resources={r"/api/*": {"origins": "*"}})

@app.route('/api/new-comment', methods=['POST'])
def welcome():
    comment = request.json['content']
    img = request.json['img']
    isYou = request.json['isYou']
    score = request.json['score']
    username = request.json['username']
    print("new: ", comment)
    return 'Successfully comment added!'

@app.route('/api/data/<int:id>', methods=['GET'])
def getApiData(id: int):
    escores, tscores, polarity = getData(id)
    return json.dumps({"escores": escores, "tscores": tscores, "polarity": polarity})

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Dec/2023 17:09:13] "GET /api/data/3 HTTP/1.1" 200 -


Id:  3
So far Comments:  [65, 1, 14, 53, 52, 43, 41, 70, 51, 17, 54, 45, 3]


127.0.0.1 - - [19/Dec/2023 17:09:16] "GET /api/data/8 HTTP/1.1" 200 -


Id:  8
So far Comments:  [65, 1, 14, 53, 52, 43, 41, 70, 51, 17, 54, 45, 3, 46, 36, 5, 15, 35, 9, 44, 40, 66, 4, 42, 39, 71, 11, 12, 74, 13, 47, 67, 6, 10, 16, 27, 28, 68, 75, 19, 57, 58, 23, 26, 59, 60, 34, 18, 33, 56, 61, 29, 8]


...

Id:  8
So far Comments:  [65, 1, 14, 53, 52, 43, 41, 70, 51, 17, 54, 45, 3, 46, 36, 5, 15, 35, 9, 44, 40, 66, 4, 42, 39, 71, 11, 12, 74, 13, 47, 67, 6, 10, 16, 27, 28, 68, 75, 19, 57, 58, 23, 26, 59, 60, 34, 18, 33, 56, 61, 29, 8]


127.0.0.1 - - [19/Dec/2023 19:37:11] "GET /api/data/43 HTTP/1.1" 200 -


Id:  43
So far Comments:  [65, 1, 14, 53, 52, 43]


127.0.0.1 - - [19/Dec/2023 19:37:42] "GET /api/data/35 HTTP/1.1" 200 -


Id:  35
So far Comments:  [65, 1, 14, 53, 52, 43, 41, 70, 51, 17, 54, 45, 3, 46, 36, 5, 15, 35]
